In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

model = nn.Linear(10, 2)
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("Optimizer works!")

Optimizer works!


In [13]:
import pandas as pd

# Load CSV (assuming no headers, just raw numeric rows)
df = pd.read_csv("scoop_bulls_test.csv", header=None)

print(df.shape)   # rows x columns
print(df.head())

(97, 639)
   0    1    2    3    4            5    6    7    8    9    ...  629  630  \
0    1    1    1    1    1  75712099.05    1    1    1    1  ...  0.0  0.0   
1    1    1    1    1    1  47189363.60    1    1    1    1  ...  NaN  NaN   
2    1    1    1    1    1  19763851.20    1    1    1    1  ...  NaN  NaN   
3    1    1    1    1    1  12688293.50    1    1    1    1  ...  NaN  NaN   
4    1    1    1    1    1  12056225.30    1    1    1    1  ...  NaN  NaN   

   631  632  633  634  635     636   637  638  
0  0.0  0.0  0.0  0.0  0.0  402.43  42.0  1.0  
1  NaN  NaN  NaN  NaN  NaN     NaN   NaN  NaN  
2  NaN  NaN  NaN  NaN  NaN     NaN   NaN  NaN  
3  NaN  NaN  NaN  NaN  NaN     NaN   NaN  NaN  
4  NaN  NaN  NaN  NaN  NaN     NaN   NaN  NaN  

[5 rows x 639 columns]


In [14]:
import numpy as np

MAX_LEN = 640

# Convert each row to numpy array and pad
data = []
for row in df.values:
    row = row[~pd.isnull(row)]  # remove NaNs if any
    padded = np.pad(row, (0, MAX_LEN - len(row)), 'constant')
    data.append(padded)

data = np.array(data)
print(data.shape)  # (num_samples, 640)


(97, 640)


In [16]:
from sklearn.model_selection import train_test_split

# Example: last column is label, rest are features
# First column is the label
y = data[:, 0].astype(int)   # labels (0 or 1)
X = data[:, 1:]              # features (639 columns after padding)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [17]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Wrap in DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)


In [18]:
import torch.nn as nn
import torch.optim as optim

class SimpleModel(nn.Module):
    def __init__(self, input_dim=639, hidden_dim=128, output_dim=2):  # binary classification
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


model = SimpleModel(input_dim=639, hidden_dim=128, output_dim=len(set(y)))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [19]:
for epoch in range(10):  # 10 epochs
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

IndexError: Target 1 is out of bounds.